# Alexnet

### 1. 介绍
Alexnet是2012年提出来的用于图像识别的神经网络架构，把深度学习模型在Imagenet比赛中的正确率提高到前所未有的高度，推动了深度学习的发展。

论文链接： https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

### 2. 创新点

#### Relu激活函数
在Alexnet之前，常用的激活函数一般是tanh或者sigmoid，这两个函数存在梯度消失的问题，Alexnet使用Relu激活函数，一定程度上缓解了梯度消失问题。

Relu = max(0, x)

#### Dropout层
无论是机器学习还是深度学习，如何避免过拟合始终是一个重要话题，Alexnet中成功地使用Dropout层以缓解过拟合，该层主要是与全连接层配合使用（因为全连接层参数量大），作用是以某种概率随机令一定的神经元失活，Alexnet中Dropout概率参数设置为0.5，即神经元经过该层都会随机失活一半数量。值得注意的是，Dropout层只在训练时使用，在测试阶段不使用。

#### 数据增强
在训练时，对每张输入图像，先进行PCA颜色扰动，然后缩放到固定大小，接着随机裁剪出一块区域，并以0.5的概率将其水平翻转，最后才送入网络。这种方法不需要增加数据，就可以有效增强网络泛化能力。


#### GPU并行运算
AlexNet在使用GPU进行训练时，将卷积层和全连接层分别放到不同的GPU上进行并行计算，从而加快了训练速度。

#### LRN局部相应归一化
对于每个特征图（卷积层后的输出）的每个位置，计算该位置周围像素的平方和，然后将该位置的像素除以这个和。一定程度上可以避免过拟合，但现在更常用的技术是Batch Normalization。

### 3. 模型架构与Pytorch代码展示

该代码展示的是假设在一个GPU上训练的模型架构，与原论文存在一定差异。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Alexnet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        
        self.lrn = nn.LocalResponseNorm(size=5, alpha=0.00001, beta=0.75, k=1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # 卷积层
        x = F.relu(self.conv1(x))
        x = self.lrn(self.maxpool(x))
        x = F.relu(self.conv2(x))
        x = self.lrn(self.maxpool(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.maxpool(F.relu(self.conv5(x)))
        
        # 全连接层
        x = torch.flatten(x, 1)
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        x = self.fc3(x)
        
        return x